In [ ]:
!pip install transformers datasets scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
from datasets import Dataset

In [ ]:
# Connec to the drive
from google.colab import drive
drive.mount('/content/drive')
# Paths
BASE_PATH = '/content/drive/MyDrive/'
DATA_SET_PATH = BASE_PATH + 'Capstone Project/Datasets/'
BALANCED_DATASET_PATH = DATA_SET_PATH + 'balanced_df_labeled.csv'

Mounted at /content/drive


In [ ]:
df = pd.read_csv(BALANCED_DATASET_PATH)

# URL ve Referrer
df["text"] = df["url"].fillna('') + " [SEP] " + df["referrer"].fillna('')

df = df[["text", "label"]]
df.head()


,text,label
0,/DVWA/controlpanel/ [SEP],normal
1,/DVWA/vulnerabilities/exec/ [SEP] http://192.1...,normal
2,/index.php/2-uncategorised/../..//../..//../.....,path_traversal
3,/DVWA/login.php [SEP] http://192.168.4.161/DVW...,normal
4,/index.php/component/search/?ordering=newest&s...,sql_injection


In [ ]:
labels = df["label"].unique().tolist()
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

df["label"] = df["label"].map(label2id)


In [ ]:
import json

with open('id2label.json', 'w') as f:
    f = json.dump(id2label, f)

print(id2label)
print(label2id)

{0: 'normal', 1: 'path_traversal', 2: 'sql_injection', 3: 'xss', 4: 'command_injection'}
{'normal': 0, 'path_traversal': 1, 'sql_injection': 2, 'xss': 3, 'command_injection': 4}


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"].tolist(), df["label"].tolist(), test_size=0.2, stratify=df["label"]
)


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(texts):
    return tokenizer(texts["text"], truncation=True)

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/4036 [00:00<?, ? examples/s]

Map:   0%|          | 0/1010 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-10-d243dd06509e>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: furkanpnr220 (furkanpnr220-maltepe-uni) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.005500,0.058789
2,0.001600,0.047971
3,0.001200,0.046580


TrainOutput(global_step=1515, training_loss=0.09292245327953015, metrics={'train_runtime': 361.0037, 'train_samples_per_second': 33.54, 'train_steps_per_second': 4.197, 'total_flos': 743421811037640.0, 'train_loss': 0.09292245327953015, 'epoch': 3.0})

In [ ]:
preds = trainer.predict(val_dataset)
pred_labels = preds.predictions.argmax(-1)

print(classification_report(val_labels, pred_labels, target_names=label2id.keys()))

                   precision    recall  f1-score   support

           normal       1.00      0.99      0.99       600
   path_traversal       0.92      1.00      0.96        59
    sql_injection       0.99      1.00      0.99       188
              xss       1.00      1.00      1.00       142
command_injection       0.90      0.90      0.90        21

         accuracy                           0.99      1010
        macro avg       0.96      0.98      0.97      1010
     weighted avg       0.99      0.99      0.99      1010



In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

model_path = "/content/results/checkpoint-1515"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)


In [ ]:
test_samples = [
    # NORMAL
    {"url": "/index.html", "referrer": "https://www.google.com"},
    {"url": "/products/view?id=42", "referrer": "https://trustedpartner.com"},
    {"url": "/blog/python-tips", "referrer": "https://facebook.com"},

    # SQL INJECTION
    {"url": "/login?username=admin' OR '1'='1&password=1234", "referrer": "https://evil.com"},
    {"url": "/search?query=abc'; DROP TABLE users; --", "referrer": "https://attacker.site"},
    {"url": "/api/user?id=1%20UNION%20SELECT%20null,null--", "referrer": "https://test123.com"},

    # XSS
    {"url": "/comments?msg=<script>alert('Hacked!')</script>", "referrer": "https://someblog.com"},
    {"url": "/feedback?input=%3Cimg%20src%3Dx%20onerror%3Dalert('xss')%3E", "referrer": "https://weirdsite.net"},
    {"url": "/profile?bio=<svg/onload=alert(123)>", "referrer": "https://fake-social.com"},

    # PATH TRAVERSAL
    {"url": "/view?file=../../../../etc/passwd", "referrer": "https://testhost.com"},
    {"url": "/readme?doc=../../../boot.ini", "referrer": "https://some-ref.com"},
    {"url": "/log/download?name=..%2F..%2Fsecret.txt", "referrer": "https://trustedref.net"},

    # COMMAND INJECTION
    {"url": "/status?host=127.0.0.1; cat /etc/passwd", "referrer": "https://cmdtest.com"},
    {"url": "/ping?ip=8.8.8.8 && rm -rf /", "referrer": "https://admin-panel.biz"},
    {"url": "/execute?cmd=ls|whoami", "referrer": "https://danger-zone.com"}
]

texts = [sample["url"] + " [SEP] " + sample["referrer"] for sample in test_samples]



In [ ]:
import torch

# Tokenize
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

# Run the Model
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)



In [ ]:
# id2label mapping
id2label = model.config.id2label

for i, pred in enumerate(predictions):
    print(f"Girdi: {texts[i]}")
    print(f"Tahmin Edilen Sınıf: {id2label[pred.item()]}")
    print("--------")



Girdi: /index.html [SEP] https://www.google.com
Tahmin Edilen Sınıf: normal
--------
Girdi: /products/view?id=42 [SEP] https://trustedpartner.com
Tahmin Edilen Sınıf: normal
--------
Girdi: /blog/python-tips [SEP] https://facebook.com
Tahmin Edilen Sınıf: normal
--------
Girdi: /login?username=admin' OR '1'='1&password=1234 [SEP] https://evil.com
Tahmin Edilen Sınıf: sql_injection
--------
Girdi: /search?query=abc'; DROP TABLE users; -- [SEP] https://attacker.site
Tahmin Edilen Sınıf: sql_injection
--------
Girdi: /api/user?id=1%20UNION%20SELECT%20null,null-- [SEP] https://test123.com
Tahmin Edilen Sınıf: sql_injection
--------
Girdi: /comments?msg=<script>alert('Hacked!')</script> [SEP] https://someblog.com
Tahmin Edilen Sınıf: xss
--------
Girdi: /feedback?input=%3Cimg%20src%3Dx%20onerror%3Dalert('xss')%3E [SEP] https://weirdsite.net
Tahmin Edilen Sınıf: sql_injection
--------
Girdi: /profile?bio=<svg/onload=alert(123)> [SEP] https://fake-social.com
Tahmin Edilen Sınıf: xss
--------


In [ ]:
!zip -r model_checkpoint_1515.zip /content/results/checkpoint-1515

  adding: content/results/checkpoint-1515/ (stored 0%)
  adding: content/results/checkpoint-1515/model.safetensors (deflated 7%)
  adding: content/results/checkpoint-1515/vocab.txt (deflated 53%)
  adding: content/results/checkpoint-1515/tokenizer_config.json (deflated 75%)
  adding: content/results/checkpoint-1515/trainer_state.json (deflated 82%)
  adding: content/results/checkpoint-1515/training_args.bin (deflated 51%)
  adding: content/results/checkpoint-1515/optimizer.pt (deflated 27%)
  adding: content/results/checkpoint-1515/scheduler.pt (deflated 56%)
  adding: content/results/checkpoint-1515/rng_state.pth (deflated 25%)
  adding: content/results/checkpoint-1515/special_tokens_map.json (deflated 42%)
  adding: content/results/checkpoint-1515/config.json (deflated 53%)


In [ ]:
from google.colab import files
files.download("model_checkpoint_1515.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>